# 🚀 Pipeline de Modelos (LSTM, GRU, MLP) - Projeto USP-IC-SIN5006
Este notebook executa modelos de séries temporais (LSTM, GRU e MLP) sobre todas as bases localizadas na pasta `datasets` do Google Drive. 
Os resultados (previsões, gráficos e métricas) serão salvos automaticamente na pasta `outputs`.

---


In [ ]:

# 🔧 Instalação de pacotes necessários
!pip install tensorflow scikit-learn pandas matplotlib seaborn plotly --quiet

# 🔗 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 🗂️ Definir os caminhos das pastas
base_path = '/content/drive/MyDrive/USP-IC-SIN5006/'
datasets_path = base_path + 'datasets/'
outputs_path = base_path + 'outputs/'


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU
import os
import glob

# 🔎 Localizar todos os datasets CSV na pasta datasets
datasets = glob.glob(datasets_path + '*.csv')
print(f'🗂️ Bases encontradas: {datasets}')

# 🎯 Função de criação de janelas temporais
def create_sequences(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back)])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

# 🔥 Função para construir o modelo
def build_model(model_type, input_shape):
    model = Sequential()
    if model_type == 'LSTM':
        model.add(LSTM(50, activation='tanh', input_shape=input_shape))
    elif model_type == 'GRU':
        model.add(GRU(50, activation='tanh', input_shape=input_shape))
    elif model_type == 'MLP':
        model.add(Dense(64, activation='relu', input_shape=(input_shape[0],)))
        model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# 📊 Função de avaliação
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred) ** 0.5
    r2 = r2_score(y_true, y_pred)
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}

# 🚀 Loop de execução para todos os modelos e datasets
results = []
look_back = 30
model_types = ['LSTM', 'GRU', 'MLP']

for dataset_path in datasets:
    dataset_name = os.path.basename(dataset_path).replace('.csv','')
    print(f"📈 Processando dataset: {dataset_name}")

    df = pd.read_csv(dataset_path)
    values = df.iloc[:, 0].values.reshape(-1, 1)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(values)

    X, y = create_sequences(scaled, look_back)

    X_LSTM = X.reshape((X.shape[0], X.shape[1], 1))
    X_MLP = X.reshape((X.shape[0], X.shape[1]))

    for model_type in model_types:
        print(f"🔧 Treinando modelo: {model_type} para {dataset_name}")

        input_shape = (X.shape[1], 1) if model_type in ['LSTM', 'GRU'] else (X.shape[1],)
        model = build_model(model_type, input_shape)

        history = model.fit(
            X_LSTM if model_type in ['LSTM', 'GRU'] else X_MLP,
            y,
            epochs=50,
            batch_size=16,
            verbose=0
        )

        y_pred = model.predict(X_LSTM if model_type in ['LSTM', 'GRU'] else X_MLP)

        y_true_inv = scaler.inverse_transform(y)
        y_pred_inv = scaler.inverse_transform(y_pred)

        metrics = evaluate_model(y_true_inv, y_pred_inv)
        metrics['Dataset'] = dataset_name
        metrics['Model'] = model_type
        results.append(metrics)

        plt.figure(figsize=(10,5))
        plt.plot(y_true_inv, label='Real')
        plt.plot(y_pred_inv, label='Previsto')
        plt.title(f'Previsão - {dataset_name} - {model_type}')
        plt.legend()
        plot_file = outputs_path + f'plot_{dataset_name}_{model_type}.png'
        plt.savefig(plot_file)
        plt.show()

        df_out = pd.DataFrame({
            'Real': y_true_inv.flatten(),
            'Previsto': y_pred_inv.flatten()
        })
        output_name = outputs_path + f'resultado_{dataset_name}_{model_type}.csv'
        df_out.to_csv(output_name, index=False)
        print(f"✅ Resultado salvo: {output_name}")
        print(f"🖼️ Gráfico salvo: {plot_file}")

# 📑 Resultados Consolidados
df_results = pd.DataFrame(results)
print(df_results)
df_results.to_csv(outputs_path + 'Resultados_Consolidados.csv', index=False)
print("📄 Resultados consolidados salvos em 'Resultados_Consolidados.csv'")
